### Run the agent locally

This notebook will show you how to run ConverseSQLAgent locally.

__Prerequisites__:
- AWS CLI installed and configured
- IAM User and Role with permissions
    - DynamoDB (Read/Write to the DynamoDB table provided to the agent)
    - Amazon Bedrock Claude 3.5 Sonnet or Claude 3 Sonnet models
    - Secrets Manager key for RDS connections and credentials details

- Amazon RDS Database accessible from the local machine
     - The credentials (username and password) must be saved in a Secrets Manager as 'Credentials for Amazon RDS Database'


In [1]:
import sys

sys.path.append('../src/ConverseSqlAgent')

from agent import BaseAgent
from tool_groups.structured_memory import STRUCTURED_MEMORY_TOOL_GROUP
from tool_groups.sql import SQL_TOOL_GROUP

In [5]:
model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"
# model_id="anthropic.claude-3-sonnet-20240229-v1:0"


In [12]:

agent = BaseAgent(model_id=model_id,
                  memory_table_name='advtext2sql_memory_tb',
                  requests_per_minute_limit=10)

agent.add_tool_group(STRUCTURED_MEMORY_TOOL_GROUP)
agent.add_tool_group(SQL_TOOL_GROUP)

### Invoking the agent

The SQL tools provided to the agent is designed to retrieve the RDS connection details from 
AWS Secrets Manager. You must first save your username and password in AWS Secrets Manager
as RDS. Your IAM role must have permissions to retrieve from AWS Secrets Manager.

You must then provide the agent the database name and the secrets manager key to use.

Then, include your query from the user.

In [14]:
# Change these parameters
database_type = "postgres"
database_name = "adventureworks2"
secrets_manager_key = "agentSQL/adventureworks2" 

input_text = "How many employees are there in the database?"


# Feel free to adjust the prompt template
prompt_template = """
Connect to the {database_type} database {database_name} using the secrets manager key {secrets_manager_key}.

Input: {input_text}
"""

prompt = prompt_template.format(database_type=database_type, 
                                database_name=database_name,
                                secrets_manager_key=secrets_manager_key,
                                input_text=input_text)

response = agent.invoke_agent(prompt)
print(response)


User: 
Connect to the postgres database adventureworks2 using the secrets manager key agentSQL/adventureworks2.

Input: How many employees are there in the database?

Beginning execution loop
Invoking converse API
Assistant: Certainly! I'll connect to the postgres database adventureworks2 using the provided secrets manager key and find out how many employees are in the database. Let me create a plan to accomplish this task.

<thinking>
To answer this question, I need to:
1. Read my memory index to check if I have any relevant information stored.
2. Connect to the database using the provided information.
3. Identify the relevant schema and table that contains employee information.
4. Query the database to count the number of employees.
5. Provide the answer in natural language.
</thinking>

<current_plan>
<rephrased_request>Determine the total number of employees in the adventureworks2 postgres database.</rephrased_request>

<memory_tool_group_analysis>
I need to check my memory index f